In [58]:
import pandas as pd
import os
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
#!/usr/bin/env python
# coding: utf-8

import glob
import json
import os
import re
import time
import wget
import urllib.parse

In [59]:
def isNaN(string):
    return string != string

In [60]:
stndrd = pd.read_csv("/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Benchmark/MoleculesStandards.csv")
del stndrd["Unnamed: 0"]

In [61]:
stndrd

,Name,CanonicalSMILES
0,Betaine,C[N+](C)(C)CC(=O)[O-]
1,Pipecolinic acid,C1CCNC(C1)C(=O)O
2,Cysteinolic acid,C(C(CS(=O)(=O)O)N)O
3,Methionine sulfoxide,CS(=O)CCC(C(=O)O)N
4,O-acetyl-L-carnitine,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C
5,O-propanoyl-L-carnitine,CCC(=O)OC(CC(=O)[O-])C[N+](C)(C)C
6,O-butanoyl-L-carnitine,CCCC(=O)OC(CC(=O)[O-])C[N+](C)(C)C
7,Isovalerylcarnitine,CC(C)CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C
8,"N,N-dimethylarginine",CN(C)C(=NCCCC(C(=O)O)N)N


In [62]:
input_table = pd.read_csv("/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Run3_Standards/input_table.csv")
input_table

,Unnamed: 0,mzml_files,ResultFileNames,File_id
0,1,./VN_211016_acetyl_carnitine.mzML,./VN_211016_acetyl_carnitine,file_1
1,2,./VN_211016_betaine.mzML,./VN_211016_betaine,file_2
2,3,./VN_211016_butanoyl_carnitine.mzML,./VN_211016_butanoyl_carnitine,file_3
3,4,./VN_211016_cyst_acid_PRM_neg.mzML,./VN_211016_cyst_acid_PRM_neg,file_4
4,5,./VN_211016_cyst_acid_PRM_pos.mzML,./VN_211016_cyst_acid_PRM_pos,file_5
5,6,./VN_211016_isovalerylcarnitine.mzML,./VN_211016_isovalerylcarnitine,file_6
6,7,./VN_211016_methionine_sulfoxide.mzML,./VN_211016_methionine_sulfoxide,file_7
7,8,./VN_211016_pipecolate.mzML,./VN_211016_pipecolate,file_8
8,9,./VN_211016_propanoyl_carnitine.mzML,./VN_211016_propanoyl_carnitine,file_9
9,10,./VN_211016_Sc_QC_PRM_neg.mzML,./VN_211016_Sc_QC_PRM_neg,file_10


In [51]:
input_table = input_table[0:9], input_table[14]
input_table

,Unnamed: 0,mzml_files,ResultFileNames,File_id
0,1,./VN_211016_acetyl_carnitine.mzML,./VN_211016_acetyl_carnitine,file_1
1,2,./VN_211016_betaine.mzML,./VN_211016_betaine,file_2
2,3,./VN_211016_butanoyl_carnitine.mzML,./VN_211016_butanoyl_carnitine,file_3
3,4,./VN_211016_cyst_acid_PRM_neg.mzML,./VN_211016_cyst_acid_PRM_neg,file_4
4,5,./VN_211016_cyst_acid_PRM_pos.mzML,./VN_211016_cyst_acid_PRM_pos,file_5
5,6,./VN_211016_isovalerylcarnitine.mzML,./VN_211016_isovalerylcarnitine,file_6
6,7,./VN_211016_methionine_sulfoxide.mzML,./VN_211016_methionine_sulfoxide,file_7
7,8,./VN_211016_pipecolate.mzML,./VN_211016_pipecolate,file_8
8,9,./VN_211016_propanoyl_carnitine.mzML,./VN_211016_propanoyl_carnitine,file_9


In [63]:
input_dir = "/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Run3_Standards"

In [65]:
# entry is all files and folders in input_dir
for entry in os.listdir(input_dir):
    # if the entry is also a directory
    if os.path.isdir(os.path.join(input_dir, entry)):
        sub_dir = input_dir + "/" + entry + "/insilico/SIRIUS/"
        msp_csv = input_dir + "/" + entry + "/insilico/MS1DATA.csv"
        if os.path.exists(msp_csv) and os.path.exists(sub_dir):
            # output json files from SIRIUS
            files_S = glob.glob(sub_dir + "/*.json")
            # list of precursor m/z
            msp = pd.read_csv(msp_csv)

            # for each mz
            for mz, row in msp.iterrows():
                # make a list of files with this mz
                files_for_mz = []

                for file in files_S:
                    if str(msp["premz"][mz]) in file:
                        files_for_mz.append(file)
                #print(files_for_mz)
                print("++++++++++++++++++++++++++++++++++++++++++++++++")
                # extract the formula and structure files
                json_dirALL = next(os.walk(files_for_mz[0]))[1]
                sub_sub_dirALL_structure_can = (
                    files_for_mz[0]
                    + "/"
                    + json_dirALL[0]
                    + "/structure_candidates.tsv"
                )
                if (
                    os.path.exists(sub_sub_dirALL_structure_can)
                    and len(pd.read_csv(sub_sub_dirALL_structure_can, sep="\t")) > 0
                ):
                    ALL_structure_csv = pd.read_csv(
                            sub_sub_dirALL_structure_can, sep="\t"
                        )
                    top_sirius = ALL_structure_csv[0:21]
                    for i, row in top_sirius.iterrows():
                        for s, row in stndrd.iterrows():
                            if (
                                not isNaN(stndrd["CanonicalSMILES"][s])
                            ):
                                LHms2 = [
                                        Chem.MolFromSmiles(
                                            top_sirius["smiles"][i]
                                        ),
                                        Chem.MolFromSmiles(
                                            stndrd["CanonicalSMILES"][s]
                                        ),
                                    ]
                                LHfps2 = [
                                        AllChem.GetMorganFingerprintAsBitVect(
                                            x2, 2, nBits=2048
                                        )
                                        for x2 in LHms2
                                    ]
                                LHtn2 = DataStructs.FingerprintSimilarity(
                                        LHfps2[0], LHfps2[1]
                                    )
                                if LHtn2 >= 1:
                                    print(i)
                                    print(entry)
                                    print(top_sirius["smiles"][i])
                                    print("++++++++++++++++++++++++++++++++++++++++++++++++")

++++++++++++++++++++++++++++++++++++++++++++++++
0
VN_211016_cyst_acid_PRM_neg
C(C(CS(=O)(=O)O)N)O
++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++
2
VN_211016_Sc_QC_PRM_neg
C(C(CS(=O)(=O)O)N)O
++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++
0
VN_211016_methionine_sulfoxide
CS(=O)CCC(C(=O)O)N
++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++
0
VN_211016_pipecolate
C1CCNC(C1)C(=O)O
++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++

In [42]:
top_sirius = ALL_structure_csv[0:21]
top_sirius

,rank,formulaRank,CSI:FingerIDScore,molecularFormula,adduct,InChIkey2D,InChI,name,smiles,xlogp,pubchemids,links,dbflags
0,1,1,-59.588562,C3H9NO4S,[M - H]-,IISXMZMYCYNVES,"InChI=1S/C3H9NO4S/c4-3(1-5)2-9(6,7)8/h3,5H,1-2...",Cysteinolic acid,C(C(CS(=O)(=O)O)N)O,-4.700000,167538;9812831;10219556;134928567,COCONUT:(CNP0250867);Natural Products:(UNPD155...,3178566
1,2,1,-64.186260,C3H9NO4S,[M - H]-,BZVKOCJZTCLEMW,"InChI=1S/C3H9NO4S/c4-1-3(5)2-9(6,7)8/h3,5H,1-2...",3-azaniumyl-2-hydroxypropane-1-sulfonate,C(C(CS(=O)(=O)O)O)N,-4.700000,5239185;5239186;57788531,PubChem:(5239185 5239186 57788531);KEGG Mine,8589934594
2,3,1,-67.419003,C3H9NO4S,[M - H]-,HFSGXAGEUWTIQO,"InChI=1S/C3H9NO4S/c4-3(5)1-2-9(6,7)8/h3,5H,1-2...",NaN,C(CS(=O)(=O)O)C(N)O,-1.767616,NaN,KEGG Mine,8589934592
3,4,1,-77.675704,C3H9NO4S,[M - H]-,FOKJSZQFVDMGGT,"InChI=1S/C3H9NO4S/c5-4-2-1-3-9(6,7)8/h4-5H,1-3...","1-Propanesulfonic acid, 3-(hydroxyamino)-",C(CNO)CS(=O)(=O)O,-3.700000,10397038,PubChem:(10397038);KEGG Mine,8589934594
4,5,1,-80.709567,C3H9NO4S,[M - H]-,CPTLGCCSZNALBZ,"InChI=1S/C3H9NO4S/c4-3(1-2-5)9(6,7)8/h3,5H,1-2...",NaN,C(CO)C(N)S(=O)(=O)O,-4.300000,53733817,PubChem:(53733817),2
5,6,1,-83.941541,C3H9NO4S,[M - H]-,UMTBGMXFTSIKBH,"InChI=1S/C3H9NO4S/c4-1-3(2-5)9(6,7)8/h3,5H,1-2...","2-Propanesulfonic acid, 1-amino-3-hydroxy-",C(C(CO)S(=O)(=O)O)N,-4.700000,22621462;101765141,PubChem:(22621462 101765141),2
6,7,1,-89.132364,C3H9NO4S,[M - H]-,PRQWYJOLLWUKNN,"InChI=1S/C3H9NO4S/c4-2-1-3(5)9(6,7)8/h3,5H,1-2...",NaN,C(CN)C(O)S(=O)(=O)O,-1.771248,NaN,KEGG Mine,8589934592
7,8,1,-91.362204,C3H9NO4S,[M - H]-,JBOWGOOXLSDXHE,"InChI=1S/C3H9NO4S/c5-2-1-4-3-9(6,7)8/h4-5H,1-3...","Methanesulfonic acid, [(2-hydroxyethyl)amino]-",C(CO)NCS(=O)(=O)O,-4.200000,20307495,PubChem:(20307495),2
8,9,1,-94.773510,C3H9NO4S,[M - H]-,WUHXNMJSNJXJEZ,"InChI=1S/C3H9NO4S/c5-3-4-1-2-9(6,7)8/h4-5H,1-3...","Ethanesulfonic acid, 2-(hydroxymethylamino)-",C(CS(=O)(=O)O)NCO,-4.200000,20265883,PubChem:(20265883);PubMed,66
9,10,1,-96.167642,C3H9NO4S,[M - H]-,LZOFQGWJBGZHMW,"InChI=1S/C3H9NO4S/c4-2-1-3-8-9(5,6)7/h1-4H2,(H...",3-aminopropyl hydrogen sulfate,C(CN)COS(=O)(=O)O,-3.700000,294069,HMDB:(140100);PubChem:(294069);PubChem class -...,16777226


In [43]:
for i, row in top_sirius.iterrows():
    for s, row in stndrd.iterrows():
        if (
            not isNaN(stndrd["CanonicalSMILES"][s])
        ):
            LHms2 = [
                    Chem.MolFromSmiles(
                        top_sirius["smiles"][i]
                    ),
                    Chem.MolFromSmiles(
                        stndrd["CanonicalSMILES"][s]
                    ),
                ]
            LHfps2 = [
                    AllChem.GetMorganFingerprintAsBitVect(
                        x2, 2, nBits=2048
                    )
                    for x2 in LHms2
                ]
            LHtn2 = DataStructs.FingerprintSimilarity(
                    LHfps2[0], LHfps2[1]
                )
            if LHtn2 >= 0.99:
                print(i)

0


In [106]:
files_betaine = ["/VN_211016_Sc_QC_PRM_pos", "/VN_211016_Sc_st_PRM_pos", "/VN_211016_betaine"]
files_Pipecolinic_acid = ["/VN_211016_Sc_QC_PRM_pos", "/VN_211016_Sc_st_PRM_pos", "/VN_211016_pipecolate"]
files_Cysteinolic_acid_pos = ["/VN_211016_Sc_QC_PRM_pos", "/VN_211016_Sc_st_PRM_pos", "/VN_211016_cyst_acid_PRM_pos"]
files_Cysteinolic_acid_neg = ["/VN_211016_Sc_QC_PRM_neg", "/VN_211016_Sc_st_PRM_neg", "/VN_211016_cyst_acid_PRM_neg"]
files_Methionine_sulfoxide = ["/VN_211016_Sc_QC_PRM_pos", "/VN_211016_Sc_st_PRM_pos", "/VN_211016_methionine_sulfoxide"]
#files_nn_dimethylarginine = ["/VN_211016_Sc_QC_PRM_pos", "/VN_211016_Sc_st_PRM_pos"]
files_acetyl_L_carnitine = ["/VN_211016_Sc_QC_PRM_pos", "/VN_211016_Sc_st_PRM_pos", "/VN_211016_acetyl_carnitine"]
files_propanoyl_L_carnitine = ["/VN_211016_Sc_QC_PRM_pos", "/VN_211016_Sc_st_PRM_pos", "/VN_211016_propanoyl_carnitine"]
files_butanoyl_L_carnitine = ["/VN_211016_Sc_QC_PRM_pos", "/VN_211016_Sc_st_PRM_pos", "/VN_211016_butanoyl_carnitine"]
files_Isovalerylcarnitine = ["/VN_211016_Sc_QC_PRM_pos", "/VN_211016_Sc_st_PRM_pos", "/VN_211016_isovalerylcarnitine"]

In [147]:
list_mol_files = [files_betaine, files_Pipecolinic_acid, files_Cysteinolic_acid_pos,
                 files_Methionine_sulfoxide, 
                 files_acetyl_L_carnitine, files_propanoyl_L_carnitine,
                 files_butanoyl_L_carnitine, files_Isovalerylcarnitine]

In [148]:
input_dir = "/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/StandardSMarinoi_Data"

In [149]:
for mol in list_mol_files:
    #print(mol)
    mol_features = pd.read_csv(input_dir+mol[2]+"/insilico/MS1DATA.csv")
    #print(mol_features["id_X"][0])
    mol_mz = mol_features['premz'][0]
    #print(mol_mz)
    mol_folder_sirius = input_dir+mol[2]+"/insilico/SIRIUS"
    print(mol_folder_sirius)
    mol_folder1 = [ele for ele in os.listdir(mol_folder_sirius) if("param.json" in ele)]
    #print(mol_folder1)
    mol_folder2 = [ele for ele in os.listdir(mol_folder_sirius+"/"+mol_folder1[0]) if(mol_features["id_X"][0] in ele)]
    #print(mol_folder2)
    mol_str = pd.read_csv(mol_folder_sirius+"/"+mol_folder1[0]+"/"+mol_folder2[0]+"/structure_candidates.tsv", sep ="\t")
    #print(mol_str["smiles"])
    #print("__________________________________")
    mol_str = mol_str[0:10]
    for smiles, row in mol_str.iterrows():
        for stnd, row in stndrd.iterrows():
            if 
            ms = [Chem.MolFromSmiles(betaine_str["smiles"][smiles]), Chem.MolFromSmiles(stndrd["CanonicalSMILES"][stnd])]
            fps = [AllChem.GetMorganFingerprintAsBitVect(x,2, nBits=2048) for x in ms]
            tn = DataStructs.FingerprintSimilarity(fps[0],fps[1])
            if tn >= 0.5:

                print(tn)
    print("__________________________________")

/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/StandardSMarinoi_Data/VN_211016_betaine/insilico/SIRIUS
Betaine
Pipecolinic acid
Cysteinolic acid
Methionine sulfoxide
O-acetyl-L-carnitine
O-propanoyl-L-carnitine
O-butanoyl-L-carnitine
Isovalerylcarnitine
__________________________________
/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/StandardSMarinoi_Data/VN_211016_pipecolate/insilico/SIRIUS
Betaine
Pipecolinic acid
Cysteinolic acid
Methionine sulfoxide
O-acetyl-L-carnitine
O-propanoyl-L-carnitine
O-butanoyl-L-carnitine
Isovalerylcarnitine
__________________________________
/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/StandardSMarinoi_Data/VN_211016_cyst_acid_PRM_pos/insilico/SIRIUS
Betaine
Pipecolinic acid
Cysteinolic acid
Methionine sulfoxide
O-acetyl-L-carnitine
O-propanoyl-L-carnitine
O-butanoyl-L-carnitine
Isovalerylcarnitine
__________________________________
/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/StandardSMarinoi_Data/VN_211016_methionine_sulfoxide/insilico/SIRIUS
Betaine
Pipecoli